In [19]:
pip install ollama


Note: you may need to restart the kernel to use updated packages.


# Load model and configure model parameters

In [20]:
import ollama
from ollama import Client

#model_version = "llama3.1:8b-instruct-fp16"
num_ctx = 16384 #context window - token allowed for each message, need to increase from default value of 2048 as our messages may overflow
temperature = "0" # should be kept low for deterministic results, default value 1
top_p = "1" 
model_version = "llama3.1:70b"
modelfile='''
FROM {}
PARAMETER num_ctx {}
PARAMETER temperature {}

'''.format(model_version, num_ctx, temperature)

#model_name = "emir_" + model_version + "_t_" + "{:.2f}".format(temperature)
model_name = "emir_" + model_version
#model_name = model_version
#ollama.create(model=model_name, modelfile=modelfile)


# Get first assessment from model

In [21]:
client = Client(
  headers={}
)
messages = []


generator_system_message = {'role': 'system', 'content': """
You are a software architect. Your job is to get a list of categorized user stories, analyze them in detail and
assign a score for each architecture pattern depending on their relevance to the project and 
if it would prove useful in the implementation of the described project.

When the user gives you a list of user stories, analyze the given user stories, give the reasoning for each architecture pattern, and then print out the assigned scores.

Here are the architecture patterns you will score:
*Layered Architecture 
*Event-Driven Architecture (Pub-Sub Architecture) 
*Microkernel Architecture 
*Microservices Architecture 
*Space-Based Architecture
*Pipeline Architecture (Pipe-Filter Architecture) 
*Client-Server Architecture

Here are the descriptions for each pattern and usage examples:

*Layered Architecture
Layered Architecture organizes software into horizontal layers, each with distinct responsibilities such as presentation, business logic, and data persistence. The core idea is that each layer only interacts with the layer directly beneath it, which makes the system more manageable and easier to maintain. Changes within a single layer can often be made without affecting other layers, enhancing modifiability. It also facilitates the separation of concerns, allowing teams to focus on specific layers without impacting the entire system. However, it can introduce performance overhead if there are many layers and each request must pass through them.
Typical Usage Example:
A common example is an enterprise web application with a front-end/UI layer, a service or business logic layer, and a database access layer. For instance, many Java EE or .NET applications follow a Layered Architecture to clearly separate presentation, business, and data access concerns.

*Event-Driven Architecture (Pub-Sub Architecture)
In an Event-Driven Architecture, components communicate primarily through the production and consumption of events. Publishers emit events without knowing which subscribers will be interested in them, and subscribers react when they receive relevant events. This decouples event producers from consumers, making the system more flexible and scalable. It also allows for asynchronous processing, where components can respond to events at different times. However, debugging can be more complex due to the non-linear flow of events and the indirect communication paths.
Typical Usage Example:
A common example is a messaging system with a message broker like Apache Kafka or RabbitMQ. For instance, e-commerce platforms use an event-driven approach to process inventory updates, orders, and notifications asynchronously.

*Microkernel Architecture
Microkernel Architecture (also known as Plugin Architecture) consists of a core system, or “kernel,” that provides the fundamental services, and additional features are implemented as plug-in modules around it. The core is kept minimal and stable, with most business logic or specialized features added through extensions or plugins. This allows for easy customization and extensibility, since new modules can be added without modifying the kernel. It also improves system resilience because a faulty plugin can be disabled or replaced without bringing down the entire system. However, designing a robust microkernel can be complex as it must accommodate a wide range of future extensions.
Typical Usage Example:
Examples include operating systems like the early versions of Mac OS X (with its Mach microkernel) and Eclipse IDE, where the core platform loads various plugin modules for functionality such as language support, testing tools, or version control integrations.

*Microservices Architecture
Microservices Architecture breaks down an application into smaller, independently deployable services, each responsible for a specific business capability. Each service can be built, tested, deployed, and scaled independently, which promotes agility and faster development cycles. Teams can choose different technologies or databases for each service if needed, providing flexibility. However, this approach introduces complexity in inter-service communication, monitoring, and data consistency. DevOps, CI/CD pipelines, and robust service discovery mechanisms are usually critical for successful microservices deployments.
Typical Usage Example:
Netflix is a well-known practitioner of microservices, splitting its platform into small, specialized services for managing user data, recommendation algorithms, and video streaming, among others. Likewise, many modern e-commerce companies adopt microservices to handle catalog services, shopping cart, and payment services separately.

*Space-Based Architecture
Space-Based Architecture (also called Cloud-Based or Tuple Space Architecture) uses the concept of a “data grid” or “processing grid,” where the application components share a common distributed memory or “space.” It is designed to handle high scalability and peak loads by distributing both the data and the processing across multiple nodes. Components can write data or tasks into the space, and other components pick up the tasks or data for processing. This approach reduces bottlenecks by avoiding central databases at high transaction volumes. However, it can be more complex to implement and manage due to the distributed nature of both data and processing.
Typical Usage Example:
Retail websites dealing with heavy seasonal traffic (like Black Friday sales) might use space-based architecture to handle massive spikes in transactions. GigaSpaces is a known platform that implements this pattern to allow for in-memory data sharing and fast scaling.

*Pipeline Architecture (Pipe-Filter Architecture)
Pipeline Architecture, also known as Pipe-Filter, organizes processing steps into a sequence of filters, where the output of one filter is the input to the next. Each filter performs a distinct operation, and data flows continuously through the pipeline. This makes the system modular, as filters can be swapped or reordered without changing the surrounding components. It is particularly suitable for data processing or transformation workflows. However, it might not be optimal for interactive applications needing bidirectional communication.
Typical Usage Example:
A classic example is a data processing workflow in ETL (Extract, Transform, Load) operations, where you have steps for extracting raw data, cleaning and transforming it, and finally loading it into a target system or database.

*Client-Server Architecture
Client-Server Architecture divides the system into two main parts: clients that request services or resources, and a server that processes these requests and returns a response. This structure simplifies the way users interact with shared data or centralized processes. The server can be scaled up independently, and security or permissions can be centrally managed. However, if the server fails or becomes overloaded, clients will be unable to access the service. It remains a fundamental design pattern for many networked applications.
Typical Usage Example:
Any typical web application employs a client-server setup: a web browser (client) connects to a remote web server to fetch and display web pages. Another example is email clients communicating with a mail server over protocols like IMAP/POP3 for receiving mail and SMTP for sending mail.


Here are the score options and their corresponding meaning:
1: "Completely unsuitable"
2: "Partially suitable"
3: "Sufficient for requirements"
4: "Well-suited"
5: "Perfectly aligned"

The final scores must be given in this format after the detailed reasoning for each architecture pattern:

**Layered Architecture**: 1
**Event-Driven Architecture (Pub-Sub Architecture)**: 4
**Microkernel Architecture**: 2
**Microservices Architecture**: 3
**Space-Based Architecture**: 1
**Pipeline Architecture (Pipe-Filter Architecture)**: 4
**Client-Server Architecture**: 3
s
"""}
messages.append(generator_system_message)


messages.append({'role': 'user', 'content': """
I will give you a list of categorized user stories created for a software project. Please analyze them in detail and give me the scoring for each architecture pattern:

User Stories:

{
    "Usability & Accessibility": [
      "As a moderator, I want to start a round by entering an item in a single multi-line text field, so that we can estimate it.",
      "As a moderator, I want to see all items we try to estimate this session, so that I can answer questions about the current story such as \"does this include ___?\".",
      "As a moderator, I want to add an item to the list of items to be estimated, so that so that we can be flexible and accommodate situations where we think of a new story while playing.",
      "As a moderator, I want to edit an item in the list of items to be estimated, so that I can make it better reflect the team's understanding of the item.",
      "As a moderator, I want to delete an item from the list of items to be estimated, so that we can remove it and not estimate it.",
      "As a moderator, I want to show all estimates immediately, so that I can decide to show the estimates that have been given even though not all estimators have given their estimate.",
      "As a moderator, I want to accept the average of all estimates, so that we can move on to the next item when we agree.",
      "As a moderator, I want to have the \"estimate\" field filled in automatically if all estimators show the same card, so that I can accept it more quickly.",
      "As a moderator, I want to enter the agreed-upon estimate, so that we can move on to the next item when we agree.",
      "As a moderator, I want to estimate a story we estimated earlier in the session again, so that we can give a new estimate if we feel different about the story after estimating other related stories.",
      "As a moderator, I want to copy/paste stories from a spreadsheet, so that I can get started more quickly.",
      "As a moderator, I want to view a transcript of a game, so that I can see the stories and estimates.",
      "As a moderator, I want to see dates and times in my local timezone, so that I don't have to do timezone conversion myself.",
      "As an estimator, I want to see the item weíre estimating, so that I know what Iím giving an estimate for.",
      "As an estimator, I want to see all items we will try to estimate this session, so that I have a feel for the sizes of the various items.",
      "As a participant, I want to immediately see that an estimate has been given by a specific estimator, so that I know who has already given an estimate and who weíre still waiting for.",
      "As a participant, I want to be able to change my estimate up until the last person selects a card and all are shown, so that I can change my mind based on information I hear.",
      "As a participant, I want to be able to see each estimator's prior estimates for the story being estimated, so that I can see how his or her opinion has changed so I can ask questions.",
      "As a participant, I want to see who gave what estimates during the current round, so that I know this when weíre discussing the estimates.",
      "As a participant, I want to be able start a two-minute countdown timer that all participants can see, so that I can limit the time spent discussing the estimates when I think weíve talked long enough.",
      "As a participant, I want to have the two-minute timer reset itself as soon as we all play an estimate, so that it's ready for use on the next round.",
      "As a participant, I want to scroll back through the stories and estimates from prior rounds, so that I can use this information when I estimate the item of the current round.",
      "As a participant, I want to always have the cards in the same order across multiple draws, so that it's easy to compare estimates.",
      "As a participant, I want to change my estimate as long as the draw has not been completed, so that I can change my mind.",
      "As a participant, I want to have a small thumbnail photo displayed near where my cards are played, so that the game is more personal because I see who I'm estimating with.",
      "As a user, I want to have the application respond quickly to my actions, so that I don't get bored.",
      "As a user, I want to be able to use Unicode, so that I can use any language I like.",
      "As a user, I want to see sensible and predictable URLs, so that the application feels logical and transparent.",
      "As a user, I want to have nice error pages when something goes wrong, so that I can trust the system and its developers."
    ],
    "Integrability & Interoperability": [
      "As a moderator, I want to import stories from a spreadsheet, so that I don't have to copy and paste each individual story.",
      "As a moderator, I want to export a transcript of a game as a HTML file, so that I can save the stories and estimates locally.",
      "As a moderator, I want to export a transcript of a game as a CSV file, so that I can further process the stories and estimates."
    ],
    "Security & Confidentiality": [
      "As a Researcher, I want results to be stored in a non-identifiable way, so that I can study the data to see things like whether estimates converged around the first opinion given by \"estimator A\" for example."
    ],
    "Manageability & Maintainability": [
      "As a moderator, I want to create a new game by entering a name and an optional description, so that I can start inviting estimators.",
      "As a moderator, I want to select an item to be estimated or re-estimated, so that the team sees that item and can estimate it.",
      "As a moderator, I want to browse through previous games, so that I can find the previous game Iím looking for.",
      "As a moderator, I want to delete a game, so that stories and estimates for this game are no longer stored.",
      "As a moderator, I want to create an account for the application by entering my name, email address, a password and a username, so that I can start using the application.",
      "As a moderator, I want to change my account details, so that I can keep my account details up-to-date.",
      "As a moderator, I want to delete my account, so that account information and games are no longer stored.",
      "As a moderator, I want to get a password reminder by email, so that I can get back to using the application when I've forgotten my password.",
      "As a moderator, I want to invite up to 15 estimators, so that we can play with large but not immense teams.",
      "As a developer, I want to have a list of definitions for commonly used terms, so that everyone working on the project can understand each other more easily."
    ],
    "Modularity & Extensibility": [
      "As a moderator, I want to invite estimators by giving them a URL where they can access the game, so that we can start the game.",
      "As a estimator, I want to join a game by entering my name on the page I received the URL for, so that I can participate.",
      "As a participant, I want to be shown all estimates at the same time after all estimators have given their estimate, so that I can be sure estimates are independent and not influenced by other estimates given in the same draw."
    ],
    "Performance Efficiency & Resource Utilization": [
      "As a developer, I want to have created database indexes, so that the queries run as fast as possible."
    ],
    "Variability & Flexibility": [
      "As a moderator, I want to select whether to have the team estimate with {0, 1/2, 1, 2, 3, 5, 8, etc.} or {0, 1, 2, 4, 8, 16, 32, etc.}, so that the team can use either the modified Fibonacci sequence or powers of 2."
    ],
    "Implementability & Ease of Development": [
      "As a developer, I want to have written a site which is compliant with XHTML and CSS standards, so that as many people as possible can access the site and view it as intended.",
      "As a developer, I want to have the application function correctly in Internet Explorer 6 and 7, Firefox 1.5 and 2, and Safari 2, so that as many people as possible can fully use the application.",
      "As a developer, I want to have the site comply with the W3C accessibility guidelines where possible, so that people with accessibility issues can use the application."
    ],
    "Exchangeability & Replaceability": [],
    "Analyzability & Traceability": [
      "As a developer, I want to be able to see some metrics on use of the game, so that I can see how much it is being used."
    ]
  }

"""})
response = client.chat(model=model_name, messages=messages)
message = response['message']
print(message['content'])
messages.append(message)


I'll analyze each user story and score them based on their relevance to various architecture patterns. Here's the scoring system:

* **Relevance**: How well does the user story align with the architecture pattern?
	+ 3: Highly relevant
	+ 2: Moderately relevant
	+ 1: Somewhat relevant
	+ 0: Not relevant

I'll score each user story against multiple architecture patterns, and then provide an overall assessment of which patterns are most relevant to the system.

Here's the analysis:

**Modularity & Extensibility**

* As a moderator, I want to invite estimators by giving them a URL where they can access the game. (Relevance: 3)
* As a estimator, I want to join a game by entering my name on the page I received the URL for. (Relevance: 3)
* As a participant, I want to be shown all estimates at the same time after all estimators have given their estimate. (Relevance: 2)

Score: 8/9

**Scalability & Performance Efficiency**

* As a developer, I want to have created database indexes, so that th

# Print log to output file

In [22]:
from datetime import datetime
import json

final_data = {
        "modelName": model_name,
        "modelParameters": modelfile,
        "selfRefinement": False,
        "oneShot": False,
        "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
        "lastMessage": messages[-1]["content"],
        "numberOfIterations": 0,
        "messages": [message["content"] for message in messages],
    }
    
# 3) Generate a filename based on model name and current timestamp
filename = f"./logs/zero-shot/log_{model_version}_{final_data['timestamp']}.json"

# 4) Write the conversation to a JSON file
with open(filename, "w", encoding="utf-8") as f:
    json.dump(final_data, f, indent=2, ensure_ascii=False)

print(f"Assesment complete. The whole conversation is saved to {filename}")

FileNotFoundError: [Errno 2] No such file or directory: './logs/zero-shot/log_llama3.1:70b_20250128_231452.json'